In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path

def get_ground_truth(args):
    (collectionName, phoneName), df = args
    
    path = root_dir / f"train/{collectionName}/{phoneName}/ground_truth.csv"
    target_df = pd.read_csv(path)
    output_df = pd.DataFrame()
    # merge derived and target by 'millisSinceGpsEpoch'
    for epoch, epoch_df in df.groupby('millisSinceGpsEpoch'):
        idx = (target_df['millisSinceGpsEpoch'] - epoch).abs().argmin()
        epoch_diff = epoch - target_df.loc[idx, 'millisSinceGpsEpoch']
        epoch_df['epoch_diff'] = epoch_diff
        epoch_df['target_latDeg'] = target_df.loc[idx, 'latDeg']
        epoch_df['target_lngDeg'] = target_df.loc[idx, 'lngDeg']
        output_df = pd.concat([output_df, epoch_df]).reset_index(drop=True)    
    return output_df

In [2]:
# metric
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [3]:
import simdkalman

# define kf model
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

In [4]:
def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in tqdm(unique_paths):
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return df

In [5]:
def linear_interpolation(input_df):
    dfs = pd.DataFrame()
    for (collectionName, phoneName), df in input_df.groupby(['collectionName','phoneName']):

        df['delta'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg'].shift(1), df['lngDeg'].shift(1))
        df['time_delta'] = df['millisSinceGpsEpoch'] - df['millisSinceGpsEpoch'].shift(1)
        df['delta'].fillna(0, inplace=True)
        df['time_delta'].fillna(0, inplace=True)
        df['speed'] = df['delta'] / (df['time_delta']/1000)  # m/s
        df['speed'].fillna(0, inplace=True)

        # 一度欠損値にする
        df.loc[50<df['speed'], ['latDeg', 'lngDeg']] = np.nan
        df['dummy_datetime'] = pd.to_datetime(df['millisSinceGpsEpoch'])
        df = df.set_index('dummy_datetime')

        # 時間に合わせて線形補間
        df = df.interpolate(method='time').reset_index(drop=True)
        dfs = pd.concat([dfs, df]).reset_index(drop=True)
    return dfs

In [6]:
# config
EXP_NAME = str(Path().resolve()).split('/')[-1]

In [7]:
EXP_NAME

'exp004'

In [8]:
# waypointを補正したdataset
root_dir = Path('../../input/')
test_df = pd.read_csv(root_dir / "baseline_locations_test.csv")
sub_df = pd.read_csv(root_dir / 'sample_submission.csv')

## LI -> KF  

In [9]:
import multiprocessing
train_df = pd.read_csv(root_dir / "baseline_locations_train.csv")
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    gr = train_df.groupby(['collectionName','phoneName'])
    dfs = pool.imap_unordered(get_ground_truth, gr)
    dfs = tqdm(dfs, total=len(gr))
    dfs = list(dfs)
train_df = pd.concat(dfs).sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch'])
train_df['distance'] = calc_haversine(train_df['latDeg'],train_df['lngDeg'],
                                      train_df['target_latDeg'], train_df['target_lngDeg'])
train_df['distance'].mean()

  0%|          | 0/73 [00:00<?, ?it/s]

3.846848374995186

In [10]:
train_df = linear_interpolation(train_df)
train_df['distance'] = calc_haversine(train_df['latDeg'],train_df['lngDeg'],train_df['target_latDeg'], train_df['target_lngDeg'])
train_df['distance'].mean()

3.6084284244593343

In [11]:
train_df = apply_kf_smoothing(train_df)
train_df['distance'] = calc_haversine(train_df['latDeg'],train_df['lngDeg'],
                                      train_df['target_latDeg'], train_df['target_lngDeg'])
train_df['distance'].mean()

  0%|          | 0/73 [00:00<?, ?it/s]

3.1564629632183157

In [12]:
# 同じcollection、epochである端末の予測値を平均する(アンサンブル的な感じ)
train_df = train_df.merge(train_df.groupby(['collectionName', 'millisSinceGpsEpoch'])[['latDeg', 'lngDeg']].mean(), on=['collectionName', 'millisSinceGpsEpoch'])
train_df = train_df.drop(['latDeg_x', 'lngDeg_x'], axis=1).rename(columns={'latDeg_y':'latDeg', 'lngDeg_y':'lngDeg'})

train_df['distance'] = calc_haversine(train_df['latDeg'],train_df['lngDeg'],
                                      train_df['target_latDeg'], train_df['target_lngDeg'])
train_df['distance'].mean()

3.1397305929214223

In [13]:
import pickle
def to_pickle(filename, obj):
    with open(filename, mode='wb') as f:
        pickle.dump(obj, f)
        
def from_pickle(filename):
    with open(filename, mode='rb') as f:
        obj = pickle.load(f)
    return obj

## test

In [14]:
test_df = linear_interpolation(test_df)

In [15]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91486 entries, 0 to 91485
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   collectionName              91486 non-null  object 
 1   phoneName                   91486 non-null  object 
 2   millisSinceGpsEpoch         91486 non-null  int64  
 3   latDeg                      91486 non-null  float64
 4   lngDeg                      91486 non-null  float64
 5   heightAboveWgs84EllipsoidM  91486 non-null  float64
 6   phone                       91486 non-null  object 
 7   delta                       91486 non-null  float64
 8   time_delta                  91486 non-null  float64
 9   speed                       91486 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 7.0+ MB


In [16]:
test_df = apply_kf_smoothing(test_df)

  0%|          | 0/48 [00:00<?, ?it/s]

In [19]:
# 同じcollection、epochである端末の予測値を平均する(アンサンブル的な感じ)
test_df = test_df.merge(test_df.groupby(['collectionName', 'millisSinceGpsEpoch'])[['latDeg', 'lngDeg']].mean(), on=['collectionName', 'millisSinceGpsEpoch'])
test_df = test_df.drop(['latDeg_x', 'lngDeg_x'], axis=1).rename(columns={'latDeg_y':'latDeg', 'lngDeg_y':'lngDeg'})

In [20]:
# sub_df = sub_df.assign(
#     latDeg = test_df['latDeg'],
#     lngDeg = test_df['lngDeg']
# )
sub_df = sub_df.drop(['latDeg', 'lngDeg'], axis=1).merge(test_df[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['phone', 'millisSinceGpsEpoch'])
sub_df.to_csv('submission.csv', index=False)

In [21]:
_sub_df = pd.read_csv('../exp002/submission.csv')

In [22]:
calc_haversine(sub_df['latDeg'],sub_df['lngDeg'],
               _sub_df['latDeg'], _sub_df['lngDeg']).mean()

0.07079483160052207